<a href="https://colab.research.google.com/github/alinnman/lineofsight/blob/main/statue_of_liberty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sample for line of sight
This sample calculates the line of sight from a target (lighthouse/mountain/skyscraper) to two observers located at various heights over the sea surface. All **heights** are specified in meters. Calculations are made without and with refraction. For refraction **air pressure** (kPa), **temperature** (degrees Celsius) and **temperature gradient** (degrees Celsius / meter) must be entered. Default values are pre-set.

A calculation for an observer at sea level is also made, and a check is made to see if the parabola approximation ("8 inches per miles squared") is correct or not.

A calculation of obstructed height is also made, and presented as a separate table.

Instructions for use: Fill in the parameters and **Press Ctrl-F9 or the arrow button below** to calculate the result which will be presented in the bottom cell.

*The pre-set values are taken from [the view](https://maps.app.goo.gl/QEKNpBXzX8unTxtp7) of Puig Major (1436) on Mallorca Island from Penyagolosa (1813 m) on the Spanish mainland.*

Allow some time for the first execution (Google Colab needs some time to start up).<br/>
When running for the first time you will get a warning about the code not originating from Google Colab (it is stored in GitHub). You can safely ignore this warning, but you may of course read the source code. Click "Show code" to inspect the source code (you may also modify).

*Tip: To simulate severe refraction set the temperature gradient to 0.1 (i.e strong temperature inversion)*.

The used maths (formulas etc.) is presented in the bottom cell.

In [17]:
from math import sqrt, pi, atan2, cos, acos
from IPython.display import Markdown, display

# @markdown ### Enter parameters:
TARGET_HEIGHT = 5000000 # @param {"type":"integer"}
TARGET_DISTANCE = 19318000 # @param {"type":"integer"}
#TARGET_DISTANCE = 6318000 # @param {"type":"integer"}
OBSERVER_HEIGHT_1 = 1000000 # @param {"type":"integer"}
OBSERVER_HEIGHT_2 = 15000000 # @param {"type":"integer"}
PRESSURE = 101 # @param {"type":"integer"}
TEMPERATURE = 20 # @param {"type":"integer"}
TEMP_GRADIENT = -0.01 # @param {"type":"number"}

EARTH_CIRCUMFERENCE_EQUATORIAL = 40075.017
EARTH_CIRCUMFERENCE_MERIDIONAL = 40007.86
EARTH_CIRCUMFERENCE = (EARTH_CIRCUMFERENCE_EQUATORIAL + EARTH_CIRCUMFERENCE_MERIDIONAL) / 2
EARTH_RADIUS = EARTH_CIRCUMFERENCE / (2 * pi)

R = EARTH_RADIUS*1000

def visibility_limit (RR : float, h1 : float, h2 : float,\
                      get_obscurations : bool = False,
                      target_distance = None)\
    -> tuple [float, float, list[tuple[float,float]]]:
  ''' Geometry for line-of-sight '''
  x1a = sqrt (((RR + h1)**2) - RR**2)
  x1r = atan2 (x1a, RR)
  x1 = x1r * RR
  x2a = sqrt (((RR + h2)**2) - RR**2)
  x2r = atan2 (x2a, RR)
  x2 = x2r * RR

  obscurations = list()
  output_target_done = False

  # Do split and get obscured height
  if get_obscurations and target_distance is not None:
    for i in range (10 + 1):
      s = x1r * ((10 - i) / 10)
      ho = RR / cos (s) - RR
      dist = x2 + s*RR
      if dist < target_distance and not output_target_done:
        dist2 = target_distance - x2
        angle = dist2 / RR
        hoPrim = RR / cos (angle) - RR
        obscurations.append ((target_distance, hoPrim, True))
        output_target_done = True
      obscurations.append ((dist, ho, False))
    if not output_target_done:
      obscurations.append ((target_distance, 0.0, True))

  return x1, x2, obscurations

def reportObsc (obsc : list[tuple[float,float, bool]]) :
  display (Markdown ("### Obscured heights (with refraction)"))
  tableString = "| Distance (m) | Obstructed Height (m) | Visible part (%) |\n"
  tableString += "| ----- | ----- | ----- |\n"

  for e in obsc:
    distance = e[0]
    distanceString = str(round(distance))
    obscHeight = e[1]
    obscTarget = e[2]

    if obscHeight == 0 and distance > 0:
      distanceString = distanceString + " and closer"
    fattyMarker = ""
    if obscTarget:
      fattyMarker = "**"
    if obscHeight < 0:
      percent = 0.0
      obscHeight = float(TARGET_HEIGHT)
    else:
      percent = 100.0 - (obscHeight / TARGET_HEIGHT) * 100.0
    if percent < 0:
      percent = 0.0
    elif percent > 100:
      percent = 100.0
    tableString += "|" + fattyMarker + distanceString + fattyMarker +\
                   "|" + fattyMarker + str(round(obscHeight,2)) + fattyMarker + \
                   "|" + fattyMarker + str(round(percent,2)) + fattyMarker + "|\n"
  display (Markdown(tableString))

def get_dip_of_horizon (hm : int | float, RR : float)\
      -> float:
    the_dip = acos (RR/(RR+hm))*(180/pi)*60
    return the_dip

DT_DH = TEMP_GRADIENT # Temperature shift with increasing elevation
K_FACTOR = 503*(PRESSURE*10)*(1/((TEMPERATURE+273)**2))*(0.0343 + DT_DH)
Ra = R / (1 - K_FACTOR) # This is the radius adjusted for refraction
# See https://en.wikipedia.org/wiki/Atmospheric_refraction#cite_note-Hirt2010-28

h1 = TARGET_HEIGHT # Height of target

display (Markdown ("# Results"))
display (Markdown ("HEIGHT of TARGET = " + str(h1) + " meters"))

for h2 in [0, OBSERVER_HEIGHT_1, OBSERVER_HEIGHT_2]: # Try some different observer elevations

  display (Markdown ("### CALCULATING for OBSERVER HEIGHT = " + str(h2) + " meters"))

  ## No refraction

  x1, x2, obsc = visibility_limit (R, h1, h2)
  distance = x1 + x2
  dipOfHorizon = get_dip_of_horizon (h2, R)
  lightHouseTable = "|Refraction|Line of Sight (meters)|Dip of Horizon (arcminutes)\n"
  lightHouseTable += "|-----|-----|-----|\n"
  lightHouseTable += "|No|" + str(round(distance))+ "|" +str(round(dipOfHorizon))+ "\n"

  # For zero elevation (observer at sea level) we can double-check with the parabola approximation
  # NOTE: The parabola approximation is useless if observer is above sea level!
  if h2 == 0:
    outputStr = "*Checking the 8 inches per miles squared approximation*<br/>"
    MILES_PER_KM = 0.621371
    miles = (distance/1000) * MILES_PER_KM
    inches = 8 * (miles**2)
    INCHES_PER_METER = 39.3701
    meters_from_inches = inches / INCHES_PER_METER
    outputStr += "*Parabola approximation of lighthouse height = " + str(round(meters_from_inches,5)) + " meters, from a distance of " +str(round(distance))+" meters.*<br/>"
    outputStr += "*.. which is " + \
                 str(abs(round(((meters_from_inches - h1) / h1),6)*100)) + \
                 " % off the real value*"
    display (Markdown (outputStr))

  ## Now adjust for refraction

  x1,x2,obsc = visibility_limit (Ra, h1, h2, get_obscurations=True, target_distance=TARGET_DISTANCE)
  distance = x1 + x2
  dipOfHorizon = get_dip_of_horizon (h2, Ra)

  lightHouseTable += "|Yes|" + str(round(distance))+ "|" +str(round(dipOfHorizon))+ "|\n"
  display (Markdown(lightHouseTable))
  if obsc is not None:
    reportObsc (obsc)


# Results

HEIGHT of TARGET = 5000000 meters

### CALCULATING for OBSERVER HEIGHT = 0 meters

*Checking the 8 inches per miles squared approximation*<br/>*Parabola approximation of lighthouse height = 3035075.02811 meters, from a distance of 6219737 meters.*<br/>*.. which is 39.2985 % off the real value*

|Refraction|Line of Sight (meters)|Dip of Horizon (arcminutes)
|-----|-----|-----|
|No|6219737|0
|Yes|6918965|0|


...

| Distance (m) | Obstructed Height (m) | Visible part (%) |
| ----- | ----- | ----- |
|**19318000**|**5000000.0**|**0.0**|
|10578291|5000000.0|0.0|
|9886395|3666425.8|26.67|
|9194498|2669902.1|46.6|
|8502602|1911566.18|61.77|
|7810705|1329493.34|73.41|
|7118809|883300.11|82.33|
|6426912|545944.93|89.08|
|5735016|299112.6|94.02|
|5043119|130513.49|97.39|
|4351223|32274.82|99.35|
|3659326 and closer|0.0|100.0|


### CALCULATING for OBSERVER HEIGHT = 15000000 meters

|Refraction|Line of Sight (meters)|Dip of Horizon (arcminutes)
|-----|-----|-----|
|No|14300554|4359
|Yes|16094475|4238|


### Obscured heights (with refraction)

| Distance (m) | Obstructed Height (m) | Visible part (%) |
| ----- | ----- | ----- |
|**19318000**|**28578371.37**|**0.0**|
|16094475|5000000.0|0.0|
|15402579|3666425.8|26.67|
|14710682|2669902.1|46.6|
|14018786|1911566.18|61.77|
|13326889|1329493.34|73.41|
|12634993|883300.11|82.33|
|11943096|545944.93|89.08|
|11251200|299112.6|94.02|
|10559303|130513.49|97.39|
|9867407|32274.82|99.35|
|9175510 and closer|0.0|100.0|


<br/><br/><br/><br/>

## Math formulas

<br/>

#### The **line of sight** $d_L$ is calculated using this formula

$d_L = R_r \times \left( \arctan\frac{\sqrt{(R_r+h_1)^2 - {R_r}^2}}{R_r} + \arctan\frac{\sqrt{(R_r+h_2)^2 - {R_r}^2}}{R_r} \right)$

Where:

$R_r$ is Earth radius. (It may be a value corrected for refraction)<br/>
$h_1$ is Target elevation<br/>
$h_2$ is Observer elevation

<br/>

#### The **obscured height** $h_o$ is calculated using this formula

$h_o = \frac{R_r}{\cos{\left(\frac{d_t - \left( \arctan{\frac{\sqrt{{\left(R_r + h_2\right)}^2-R_r^2}}{R_r}} \times R_r \right)}{R_r}\right)}} - R_r$

Where:

$d_t$ is the **target distance**

<br/>

#### **Refraction** is calculated using an enlarged Earth radius $R_a$ according to this formula

$R_a = \frac{R_r}{1 - 503 \times P \times \frac{1}{{\left( T+273 \right) }^2} \times \left( 0.0343 + \frac{dT}{dH}\right)}$

Where:

$P$ is Air Pressure (in millibars)<br/>
$T$ is Temperature (in degrees Celsius)<br/>
$\frac{dT}{dH}$ is Temperature Gradient (degrees celsius per meter). Normal conditions is about 1 degree colder for each 100 m, i.e. $-0.01$.

<br/>

#### **Dip of the Horizon**

$a_{\text{dip}} = \arccos{\frac{R_r}{R_r + h_2}}$